In [1]:
# importing general packages

import numpy as np
import sys
import os
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# importing additional packages

from scipy import integrate
import xspec as xs
xs.Xset.allowPrompting = False # keeps pyxspec from hanging, waiting for a response to a prompt

### Sanity check:
Idea: take the lowest normalization (3.68e-52 cm^-2); integrate over the ALP spectrum over time & energy, to get fluence. How does it compare to the fluences reported in the catalog?

#### GRB 080825 C
8 sigma detection in LLE: GRB 080825C: https://arxiv.org/pdf/0910.4192.pdf
LLE analysis spectral fit given below:

In [2]:
xs.AllData.clear()

# GBM Data

# NaI

s1=xs.Spectrum("bn080825593_n0_srcspectra.pha{*}")
s1.response="bn080825593_n0_weightedrsp.rsp"
s1.background="bn080825593_n0_bkgspectra.bak{1}"

s2=xs.Spectrum("bn080825593_n1_srcspectra.pha{*}")
s2.response="bn080825593_n1_weightedrsp.rsp"
s2.background="bn080825593_n1_bkgspectra.bak{1}"

s3=xs.Spectrum("bn080825593_n2_srcspectra.pha{*}")
s3.response="bn080825593_n2_weightedrsp.rsp"
s3.background="bn080825593_n2_bkgspectra.bak{1}"


# BGO

s4=xs.Spectrum("bn080825593_b1_srcspectra.pha{*}")
s4.response="bn080825593_b1_weightedrsp.rsp"
s4.background="bn080825593_b1_bkgspectra.bak{1}"

# LAT data

# LLE

s5=xs.Spectrum("bn080825593_LAT-LLE_srcspectra.pha{*}")
s5.response="bn080825593_LAT-LLE_weightedrsp.rsp"
s5.background="bn080825593_LAT-LLE_bkgspectra.bak{1}"

# LAT

s6=xs.Spectrum("gll_ft1_tr_bn080825593_v00_filt_spec_0.000_25.000.pha")
s6.response="gll_ft1_tr_bn080825593_v00_filt_spec_0.000_25.000.rsp"
s6.background="gll_ft1_tr_bn080825593_v00_filt_spec_0.000_25.000.bak"


xs.AllData.ignore("1-3: **-8.0")
xs.AllData.ignore("4: **-200.")
xs.AllData.ignore("4: 40e3-**")
xs.AllData.ignore("5: **-20e3")
xs.AllData.ignore("bad")

In [3]:
xs.AllModels.clear()
m = xs.Model("grbm")
xs.Fit.statMethod = "cstat 1-4"
xs.Fit.statMethod = "pgstat 5-6"
xs.Fit.perform()

In [4]:
norm = m.grbm.norm.values[0]
alpha = m.grbm.alpha.values[0]
beta = m.grbm.beta.values[0]
tem = m.grbm.tem.values[0]
red_chi2 = xs.Fit.statistic/xs.Fit.dof
print("Reduced chi^2 for this fit is %.2f." %red_chi2)


def get_parVals():
    pars = [alpha, beta, tem, norm]    
    return pars
print(get_parVals())

Reduced chi^2 for this fit is 1.33.
[0.4040647769508021, -2.283940300339788, 46.236233074964325, 0.2637855165696815]


In [5]:
# Defining the Band function: https://ui.adsabs.harvard.edu/abs/1993ApJ...413..281B/abstract
# in XSPEC: https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/node181.html

def band(energy):
    '''
    BAND(energy, alpha, beta, tem, norm) 
        
    PARAMETERS:
    ----------
    energy: float, energy element from an array of energies over which 
            the function is integrated over. 
    alpha:  float, alpha (low-energy index) parameter value.  
    beta:   float, beta (high-energy index) parameter value.     
    tem: float, characteristic energy parameter value.  
    norm:   float, normalization (aka amplitude) parameter value.  
    ''' 
    alpha, beta, tem, norm = get_parVals()

    a = float(alpha)
    b = float(beta)
    t = float(tem)
    N = float(norm)
    eng  = energy
    
    if eng < ( (a-b) * t ):
        return  N * (((eng/100.0)**a) * (np.exp(-eng/t)))
    else:
        return  N * (((((a-b)*t)/100.0)**(a-b)) * (np.exp(b-a))*((eng/100.0)**b))

def eband(energy):
    eng  = energy
    return eng * band(eng)

In [6]:
keVtoerg    = 1.60217657E-9 # conversion factor from keV to erg
emin        = 15000. # in keV
emax        = 350000. 

In [7]:
Flux_Ph = integrate.quad(band, emin, emax, limit=100)[0]
Flux_En = integrate.quad(eband, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t cts s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  0.003957792 	 cts s^-1 cm^-2
Energy Flux:  2.587849884e-07 	 ergs s^-1 cm^-2



### Normalized ALP Spectrum

In [8]:
# importing ALP packages

sys.path.append('/Users/milena/Desktop/2nd_yr_project/ALPs/ALPs_analysis/') # path to calc_alp_signal script
from calc_alp_signal import ALPSNSignal

In [9]:
EMeV = np.linspace(15.,350,100) # [MeV]
ts = np.linspace(0.1,25.,250)
ee, tt = np.meshgrid(EMeV,ts, indexing = 'ij')
alp_sn10 = ALPSNSignal(Mprog = 10.) # in 10^52
dndedt_alp10 = alp_sn10(EMeV, ts, g10=0.1) # Flux, [10^52 ph/(MeV s)]

SED10 = integrate.simps(dndedt_alp10, ts)

In [10]:
# lowest normalization for which we are sensitive to ALPs, medium background, P=0.1
norm = 3.68 # [cm^-2], in  10^-52
fluence = norm*integrate.simps(SED10, EMeV)/25. #averaging over time elapsed
print("The fluence of the ALP signal corresponding to normalization 3.68e-52 is %.3f cts/ s cm^2." %fluence)

The fluence of the ALP signal corresponding to normalization 3.68e-52 is 0.039 cts/ s cm^2.


This is the value averaged over 25 seconds of the ALP emission.

Off by a factor of ~10. Why? Possible explanations:
1. GRB Band fit is governed by GBM and LAT data, not LLE. Also, LLE data extends beyond 300000 (covers LAT's energy range as well), which will contribute to the 8-sigma detection. Eg. if we extend the emax above to 3 GeV (which is LLE's energy range), the flux from the Band fit will get to 0.056 cts / s cm^2, closer to the ALP triggering value.